In [2]:
# import pyodbc
# connect_string = ''
# try:
#     conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.236.6.111;PORT=1433;DATABASE=Teambuilder;UID=admin;PWD=TeamBuilder;')
#     print("{c} is working".format(c=connect_string))
# except pyodbc.Error as ex:
#     print("{c} is not working".format(c=connect_string))


In [3]:
# import pandas as pd
# ##After connecting to database extract preprocessed information
# #Read in ID and resumes using pandas

# #This is how to create a two column list using two lists newlist = list(zip(numbers,letters))
# #Get data from server
# student_id = []
# student_resume = []
# cursor = conn.cursor()
# cursor.execute("SELECT ALL * FROM StudentInformation") #StudentInformation is the name of the table
# for row in cursor:
#         student_id.append(row.StudentName)
#         student_resume(row.Resume)
# #Store data in pandas data frame
# student_info = list(zip(student_id, student_resume))
# df = pd.DataFrame(student_info, columns=['ID', 'Processed Resume'])
# df.head()

In [4]:
#use the same techniques we used to train to train the model with student names and identites

In [5]:
#Setting Up the Environment
import re #For Preprocessing
import pandas as pd #For Data Handling
from time import time #To Time the Operations
from collections import defaultdict #Use for word frequency

import spacy #More Preprocessing

import logging #Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing

from gensim.models import Word2Vec

In [6]:
from tika import parser
import os
df = pd.DataFrame(columns=['CandidateID', 'resume', 'cleaned_resume'])
for dirname , _, filenames in os.walk('test'):
    for filename in filenames:
       path = os.path.join(dirname, filename)
       raw = parser.from_file(path)
       df = df.append({'CandidateID' : filename[:-4], 'resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'cleaned_resume' : ""}, 
                ignore_index = True)
   
df.shape
df.head()

,CandidateID,resume,cleaned_resume
0,candidate_009,CutThroat Media NLP analyst for user commen...,
1,candidate_046,HOWARD GOODMAN F R E S H E R A N D N L P ...,
2,candidate_070,"Publicis Sapient DATA PROGRAMMER, JAN 2019 -...",
3,candidate_143,M A T T H E W H O R R E S JUNIOR DEVELOPER ...,


In [7]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [8]:
df['cleaned_resume'] = df.resume.apply(lambda x: cleanResume(x))
df = df.drop('resume', 1)
df.shape
df.head()

,CandidateID,cleaned_resume
0,candidate_009,CutThroat Media NLP analyst for user comments...
1,candidate_046,HOWARD GOODMAN F R E S H E R A N D N L P E N ...
2,candidate_070,Publicis Sapient DATA PROGRAMMER JAN 2019 TIL...
3,candidate_143,M A T T H E W H O R R E S JUNIOR DEVELOPER WO...


In [9]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [10]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['cleaned_resume'])
df.head()

,CandidateID,cleaned_resume
0,candidate_009,CutThroat Media NLP analyst for user comments...
1,candidate_046,HOWARD GOODMAN F R E S H E R A N D N L P E N ...
2,candidate_070,Publicis Sapient DATA PROGRAMMER JAN 2019 TIL...
3,candidate_143,M A T T H E W H O R R E S JUNIOR DEVELOPER WO...


In [11]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.0 mins


In [12]:
df_clean = pd.DataFrame(columns=["CandidateID", "clean"])
df_clean = df[["CandidateID"]].copy()
df_clean['clean'] = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean['clean'] = df['CandidateID'] + " " + df_clean['clean']
df_clean.shape
df_clean

INFO - 23:50:43: Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 23:50:43: NumExpr defaulting to 8 threads.


,CandidateID,clean
0,candidate_009,candidate_009 cutthroat medium nlp analyst u...
1,candidate_046,candidate_046 howard goodman f r e s h e r n...
2,candidate_070,candidate_070 publicis sapient datum program...
3,candidate_143,candidate_143 m t t h e w h o r r e s junior...


In [13]:
from gensim.models.phrases import Phrases, Phraser

In [14]:
sent = [row.split() for row in df_clean['clean']]

In [15]:
phrases = Phrases(sent, min_count=5, progress_per=10000)

INFO - 23:50:43: collecting all words and their counts
INFO - 23:50:43: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 23:50:43: collected 713 token types (unigram + bigrams) from a corpus of 464 words and 4 sentences
INFO - 23:50:43: merged Phrases<713 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
INFO - 23:50:43: Phrases lifecycle event {'msg': 'built Phrases<713 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000> in 0.00s', 'datetime': '2021-11-14T23:50:43.645383', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [16]:
bigram = Phraser(phrases)

INFO - 23:50:43: exporting phrases from Phrases<713 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
INFO - 23:50:43: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<1 phrases, min_count=5, threshold=10.0> from Phrases<713 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000> in 0.00s', 'datetime': '2021-11-14T23:50:43.674411', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [17]:
sentences = bigram[sent]
print(sentences[1])

['candidate_046', 'howard', 'goodman', 'f', 'r', 'e', 's', 'h', 'e', 'r', 'n', 'd', 'n', 'l', 'p', 'e', 'n', 'g', 'n', 'e', 'e', 'r', 'professional', 'profile', 'fresher', 'absolutely', 'love', 'work', 'vaguely', 'relate', 'machine', 'learning', 'acquaint', 'state', 'art', 'nlp', 'model', 'like', 'bert', 'xlnet', 'gpt', 'work', 'datum', 'exploration', 'proficient', 'skill', 'machine_learn', 'natural', 'language', 'process', 'deep', 'learning', 'sentiment', 'analysis', 'python', 'nltk', 'gpt', 'xlnet', 'text', 'analysis', 'text', 'extraction', 'ocr', 'employment', 'history', 'nlp', 'development', 'team', 'take', 'build', 'different', 'pipeline', 'training', 'evaluation', 'different', 'textual', 'datum', 'nlp', 'developer', 'intern', 'apr', 'nov', 'zynta', 'lab', 'educational', 'history', 'b', 'tech', 'electrical', 'rajiv', 'gandhi', 'memorial', 'university', 'delhi', 'achievement', 'extra', 'curricular', 'tensorflow', 'kera', 'certify', 'project', 'embed', 'device', 'convert', 'asl', 'v

In [18]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

285

In [19]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['datum',
 'work',
 'nlp',
 'machine_learn',
 'python',
 'project',
 'r',
 'e',
 'skill',
 'analysis']

In [20]:
import multiprocessing
import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [30]:
# model = Word2Vec.load('basemodel.bin')
# model.min_count = 30
# model.alpha = 0.02
# model.window = 3

# name = []
# print(len(model.wv.vectors))
# for i in df_clean.CandidateID:
#     for j in range(30):
#         name.append(i)

#     name = bigram[name]
#     model.build_vocab(name, update=True, progress_per=30)
#     model.train(name, total_examples=name.count, epochs=30)
#     name = []
# print(len(model.wv.vectors))
documents = [TaggedDocument(doc, tags=[df_clean.CandidateID[i]]) for i, doc in enumerate(sentences)]
model = Doc2Vec(documents, dm = 0, alpha=0.025, vector_size=30, min_alpha = 0.025, min_count = 1)

INFO - 23:56:24: collecting all words and their counts
INFO - 23:56:24: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 23:56:24: collected 285 word types and 4 unique tags from a corpus of 4 examples and 457 words
INFO - 23:56:24: Creating a fresh vocabulary
INFO - 23:56:24: Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 285 unique words (100.0%% of original 285, drops 0)', 'datetime': '2021-11-14T23:56:24.731950', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 23:56:24: Doc2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 457 word corpus (100.0%% of original 457, drops 0)', 'datetime': '2021-11-14T23:56:24.731950', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 23:56:24: deleting th

In [54]:
tokens = "technology".split()

new_vec = model.infer_vector(tokens)
sims = model.docvecs.most_similar([new_vec])
sims

<ipython-input-54-f882e909280d>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([new_vec])


[('candidate_009', 0.9550108909606934),
 ('candidate_046', 0.953600287437439),
 ('candidate_070', 0.952103316783905),
 ('candidate_143', 0.9485359787940979)]